## Gradio

In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr # oh yeah!

load_dotenv(override=True)

openai = OpenAI()

messages = [
    {"role": "system", "content": "You are a helpful assistant that can answer questions and help with tasks."},
    {"role": "user", "content": "What is the capital of the moon?"}
]

api_key = os.getenv("OPENAI_API_KEY")
response = openai.chat.completions.create(
    model="gpt-4o-mini",

    messages=messages
)

print(response.choices[0].message.content)


c:\Users\maina\miniconda3\envs\llms\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The Moon does not have a capital city because it is not a sovereign entity or country. The Moon is a natural satellite of Earth and does not have an established government or administrative divisions. Any human presence on the Moon, such as missions from different countries or organizations, does not imply the existence of a capital.


In [2]:
def shout(text):
    print(f"Shout has been called with input {text}")
    return text.upper()

In [3]:
shout("hello")

Shout has been called with input hello


'HELLO'

In [6]:
gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


In [ ]:
# Adding share=True means that it can be accessed publically
# A more permanent hosting is available using a platform called Spaces from HuggingFace, which we will touch on next week
# NOTE: Some Anti-virus software and Corporate Firewalls might not like you using share=True. 
# If you're at work on on a work network, I suggest skip this test.

gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch(share=True)

In [ ]:
# Adding inbrowser=True opens up a new browser window automatically

gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch(inbrowser=True)

In [ ]:
gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch(inbrowser=True, auth=("ed", "bananas"))

### Dark Mode

In [ ]:
# Define this variable and then pass js=force_dark_mode when creating the Interface

force_dark_mode = """
function refresh() {
    const url = new URL(window.location);
    if (url.searchParams.get('__theme') !== 'dark') {
        url.searchParams.set('__theme', 'dark');
        window.location.href = url.href;
    }
}
"""
gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never", js=force_dark_mode).launch()

In [ ]:
message_input = gr.Textbox(label="Your message:", info="Enter a message to be shouted", lines=7)
message_output = gr.Textbox(label="Response:", lines=8)

view = gr.Interface(
    fn=shout,
    title="Shout", 
    inputs=[message_input], 
    outputs=[message_output], 
    examples=["hello", "howdy"], 
    flagging_mode="never"
    )
view.launch()

In [32]:
import os
from dotenv import load_dotenv
from openai import OpenAI


load_dotenv(override=True)

OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")


def message_gpt(prompt: str) -> None:
    """
    Sends a prompt to OpenRouter using the OpenAI client and prints the response.
    """
    if not OPENROUTER_API_KEY:
        raise ValueError("OPENROUTER_API_KEY is not set in environment variables.")

    client = OpenAI(
        api_key=OPENROUTER_API_KEY,
        base_url="https://openrouter.ai/api/v1",
    )

    messages = [
        {"role": "system", "content": "You are a helpful and funny assistant."},
        {"role": "user", "content": prompt},
    ]

    response = client.chat.completions.create(
        model="openai/gpt-4.1-mini",
        messages=messages,
    )
    return response.choices[0].message.content

In [ ]:
message_input = gr.Textbox(label="Your message:", info="Enter a message for GPT-4.1-mini", lines=7)
message_output = gr.Markdown(label="Response:")

view = gr.Interface(
    fn=message_gpt,
    title="GPT", 
    inputs=[message_input], 
    outputs=[message_output], 
    examples=[
        "Explain the Transformer architecture to a layperson",
        "Explain the Transformer architecture to an aspiring AI engineer",
        ], 
    flagging_mode="never"
    )
view.launch()

In [ ]:
# Let's create a call that streams back results
# If you'd like a refresher on Generators (the "yield" keyword),
# Please take a look at the Intermediate Python guide in the guides folder

def stream_gpt(prompt):
    messages = [
        {"role": "system", "content": "You are a funny assistant"},
        {"role": "user", "content": prompt}
      ]
    stream = openai.chat.completions.create(
        model='gpt-4.1-mini',
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [ ]:
message_input = gr.Textbox(label="Your message:", info="Enter a message for GPT-4.1-mini", lines=7)
message_output = gr.Markdown(label="Response:")

view = gr.Interface(
    fn=stream_gpt,
    title="GPT", 
    inputs=[message_input], 
    outputs=[message_output], 
    examples=[
        "Explain the Transformer architecture to a layperson",
        "Explain the Transformer architecture to an aspiring AI engineer",
        ], 
    flagging_mode="never"
    )
view.launch()

In [41]:
#Based on day5-brochure.ipynb in week one, I want to combine with gradio and provide UIs interface that should allow user to provide a link and a brochure pdf is generated which a user can download

### Brochure generator (URL → downloadable PDF)

In [47]:
import os
import re
import sys
import json
import tempfile
from datetime import datetime
from pathlib import Path
from urllib.parse import urljoin, urlparse

import gradio as gr
from dotenv import load_dotenv
from openai import OpenAI
from fpdf import FPDF


def _find_project_root() -> Path:
    cwd = Path.cwd()
    for p in [cwd, *cwd.parents]:
        if (p / "requirements.txt").exists():
            return p
    return cwd


PROJECT_ROOT = _find_project_root()
WEEK1_DIR = PROJECT_ROOT / "week1"
if str(WEEK1_DIR) not in sys.path:
    sys.path.append(str(WEEK1_DIR))

from scraper import fetch_website_links, fetch_website_contents  # noqa: E402

load_dotenv(override=True)

OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")
OPENROUTER_MODEL = os.getenv("OPENROUTER_MODEL", "anthropic/claude-haiku-4-5")
OPENROUTER_BASE_URL = os.getenv("OPENROUTER_BASE_URL", "https://openrouter.ai/api/v1")

client = (
    OpenAI(api_key=OPENROUTER_API_KEY, base_url=OPENROUTER_BASE_URL)
    if OPENROUTER_API_KEY
    else None
)


def _normalize_url(url: str) -> str:
    url = (url or "").strip()
    if not url:
        return ""
    if not url.startswith(("http://", "https://")):
        url = "https://" + url
    return url


def _infer_company_name(url: str) -> str:
    netloc = urlparse(url).netloc.lower()
    if netloc.startswith("www."):
        netloc = netloc[4:]
    name = netloc.split(":")[0]
    if name.endswith(('.com', '.ai', '.io', '.co', '.org', '.net')):
        name = name.rsplit(".", 1)[0]
    name = name.replace("-", " ").replace("_", " ").strip()
    return name.title() if name else "Company"


LINKS_SYSTEM_PROMPT = """
You are provided with a list of links found on a webpage.
You decide which links are most relevant to include in a brochure about the company,
such as About, Company, Product, Customers, Careers/Jobs.
Respond in JSON like:
{
  "links": [
    {"type": "about", "url": "https://example.com/about"}
  ]
}
Do not include terms/privacy pages, login pages, or mailto links.
""".strip()


def _get_links_user_prompt(url: str) -> str:
    parsed = urlparse(url)
    base = f"{parsed.scheme}://{parsed.netloc}"

    links = fetch_website_links(url)
    absolute_links: list[str] = []
    for link in links:
        absolute = urljoin(base, link)
        if absolute.startswith("http"):
            absolute_links.append(absolute)

    unique_links = list(dict.fromkeys(absolute_links))

    prompt = (
        f"Here is the list of links on the website {url}.\n"
        "Pick the few that are relevant for a company brochure and return ONLY JSON.\n\n"
        "Links:\n\n"
        + "\n".join(unique_links[:200])
    )
    return prompt


def _extract_json_object(text: str) -> str:
    text = (text or "").strip()
    if text.startswith("```"):
        parts = text.split("```")
        if len(parts) >= 2:
            text = parts[1]
            if text.lstrip().startswith("json"):
                text = text.lstrip()[4:]
    text = text.strip()
    start, end = text.find("{"), text.rfind("}")
    if start != -1 and end != -1 and end > start:
        return text[start : end + 1]
    return text


def select_relevant_links(url: str) -> list[dict]:
    if client is None:
        raise gr.Error("Missing OPENROUTER_API_KEY. Add it to your .env, then re-run this cell.")

    resp = client.chat.completions.create(
        model=OPENROUTER_MODEL,
        messages=[
            {"role": "system", "content": LINKS_SYSTEM_PROMPT},
            {"role": "user", "content": _get_links_user_prompt(url)},
        ],
    )

    raw = resp.choices[0].message.content
    json_text = _extract_json_object(raw)
    data = json.loads(json_text)

    links = data.get("links", [])
    if not isinstance(links, list):
        return []

    cleaned: list[dict] = []
    for item in links:
        if not isinstance(item, dict):
            continue
        link_url = (item.get("url") or "").strip()
        link_type = (item.get("type") or "link").strip()
        if link_url.startswith("http"):
            cleaned.append({"type": link_type, "url": link_url})
    return cleaned


def fetch_page_and_relevant_links(url: str, max_links: int = 6) -> str:
    contents = fetch_website_contents(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"

    try:
        relevant_links = select_relevant_links(url)
    except Exception:
        relevant_links = []

    for link in relevant_links[:max_links]:
        result += f"\n\n### Link: {link['type']}\n"
        try:
            result += fetch_website_contents(link["url"])
        except Exception:
            continue

    return result


BROCHURE_SYSTEM_PROMPT = """
You analyze the contents of several relevant pages from a company website
and create a concise brochure (300-400 words max) for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Use exactly one H1 (company name), H2 headings, H3 sub-headings, bullet lists, and short paragraphs.
Cover: what the company does, company culture, key customers, and careers (only if info is available).
""".strip()


def get_brochure_user_prompt(company_name: str, url: str) -> str:
    prompt = (
        f"You are looking at a company called: {company_name}\n"
        "Here are the contents of its landing page and other relevant pages;\n"
        "use this information to build a brochure in markdown.\n\n"
    )
    prompt += fetch_page_and_relevant_links(url)
    return prompt[:5_000]


def create_brochure_markdown(company_name: str, url: str) -> str:
    if client is None:
        raise gr.Error("Missing OPENROUTER_API_KEY. Add it to your .env, then re-run this cell.")

    user_prompt = get_brochure_user_prompt(company_name, url)
    resp = client.chat.completions.create(
        model=OPENROUTER_MODEL,
        messages=[
            {"role": "system", "content": BROCHURE_SYSTEM_PROMPT},
            {"role": "user", "content": user_prompt},
        ],
    )
    return (resp.choices[0].message.content or "").strip()


def _safe_latin1(text: str) -> str:
    return (text or "").encode("latin-1", errors="replace").decode("latin-1")


def _strip_md(text: str) -> str:
    text = re.sub(r"\*\*(.*?)\*\*", r"\1", text or "")
    text = re.sub(r"\*(.*?)\*", r"\1", text)
    text = re.sub(r"`(.*?)`", r"\1", text)
    text = re.sub(r"\[([^\]]*)\]\([^)]*\)", r"\1", text)
    return text.strip()


def markdown_to_pdf(md_content: str, company_name: str, output_path: Path) -> None:
    # ── Colour palette ───────────────────────────────────────────
    C_NAVY = (22, 60, 110)
    C_BLUE = (41, 98, 200)
    C_TEAL = (0, 150, 136)
    C_ORANGE = (210, 90, 20)
    C_DARK = (40, 44, 52)
    C_WHITE = (255, 255, 255)

    pdf = FPDF()
    pdf.set_margins(20, 20, 20)
    pdf.set_auto_page_break(auto=True, margin=20)
    pdf.add_page()
    W = pdf.w - pdf.l_margin - pdf.r_margin

    # Header banner
    pdf.set_fill_color(*C_NAVY)
    pdf.rect(0, 0, pdf.w, 40, style="F")
    pdf.set_xy(0, 9)
    pdf.set_font("Helvetica", "B", 26)
    pdf.set_text_color(*C_WHITE)
    pdf.cell(pdf.w, 13, _safe_latin1(company_name.upper()), align="C")
    pdf.set_xy(0, 24)
    pdf.set_font("Helvetica", "I", 11)
    pdf.set_text_color(180, 210, 255)
    pdf.cell(pdf.w, 8, _safe_latin1("Company Brochure"), align="C")
    pdf.ln(26)

    # Rule
    pdf.set_draw_color(*C_TEAL)
    pdf.set_line_width(1.0)
    pdf.line(pdf.l_margin, pdf.get_y(), pdf.w - pdf.r_margin, pdf.get_y())
    pdf.ln(6)

    for line in (md_content or "").splitlines():
        stripped = _strip_md(line)

        if line.startswith("# "):
            continue
        if line.startswith("## "):
            pdf.ln(4)
            y = pdf.get_y()
            pdf.set_fill_color(*C_BLUE)
            pdf.rect(pdf.l_margin, y, 3, 10, style="F")
            pdf.set_xy(pdf.l_margin + 6, y)
            pdf.set_font("Helvetica", "B", 14)
            pdf.set_text_color(*C_BLUE)
            pdf.multi_cell(W - 6, 10, _safe_latin1(stripped))
            pdf.set_draw_color(*C_BLUE)
            pdf.set_line_width(0.3)
            pdf.line(pdf.l_margin, pdf.get_y(), pdf.l_margin + W, pdf.get_y())
            pdf.ln(3)
            continue
        if line.startswith("### "):
            pdf.ln(2)
            pdf.set_font("Helvetica", "B", 12)
            pdf.set_text_color(*C_TEAL)
            pdf.multi_cell(W, 7, _safe_latin1(stripped))
            pdf.ln(1)
            continue
        if line.startswith(("- ", "* ", "+ ")):
            pdf.set_font("Helvetica", "B", 11)
            pdf.set_text_color(*C_ORANGE)
            pdf.set_x(pdf.l_margin)
            pdf.cell(6, 7, _safe_latin1(">"))
            pdf.set_font("Helvetica", "", 11)
            pdf.set_text_color(*C_DARK)
            pdf.multi_cell(W - 6, 7, _safe_latin1(_strip_md(line[2:])))
            continue
        if line.strip() == "---":
            pdf.ln(2)
            pdf.set_draw_color(*C_TEAL)
            pdf.set_line_width(0.4)
            pdf.line(pdf.l_margin, pdf.get_y(), pdf.l_margin + W, pdf.get_y())
            pdf.ln(4)
            continue
        if line.strip():
            pdf.set_font("Helvetica", "", 11)
            pdf.set_text_color(*C_DARK)
            pdf.multi_cell(W, 7, _safe_latin1(stripped))
            pdf.ln(1)
        else:
            pdf.ln(3)

    pdf.set_y(-14)
    pdf.set_font("Helvetica", "I", 8)
    pdf.set_text_color(150, 150, 150)
    pdf.cell(0, 6, _safe_latin1(f"{company_name}  |  AI-generated brochure"), align="C")

    pdf.output(str(output_path))


def generate_brochure_pdf(url: str, company_name: str) -> tuple[str, str]:
    url = _normalize_url(url)
    if not url:
        raise gr.Error("Please paste a URL.")
    if not urlparse(url).netloc:
        raise gr.Error("That doesn’t look like a valid URL. Example: https://huggingface.co")

    company_name = (company_name or "").strip() or _infer_company_name(url)

    try:
        md = create_brochure_markdown(company_name, url)
    except Exception as e:
        raise gr.Error(f"Failed to generate brochure text: {type(e).__name__}: {e}")

    brochures_dir = PROJECT_ROOT / "brochures"
    brochures_dir.mkdir(exist_ok=True)

    safe_name = re.sub(r"[^\w\s-]", "", company_name).strip().replace(" ", "_") or "brochure"
    ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_path = brochures_dir / f"{safe_name}_{ts}.pdf"

    try:
        markdown_to_pdf(md, company_name, output_path)
    except Exception as e:
        raise gr.Error(f"Failed to build PDF: {type(e).__name__}: {e}")

    return md, str(output_path)


with gr.Blocks(title="Brochure Generator") as brochure_app:
    gr.Markdown(
        """
        ## Company brochure generator

        Paste a company website URL. The app will create a short brochure and generate a downloadable PDF.

        **Setup**: add `OPENROUTER_API_KEY` to your `.env`.
        """
    )

    with gr.Row():
        url_in = gr.Textbox(label="Company URL", placeholder="https://huggingface.co")
        name_in = gr.Textbox(label="Company name (optional)", placeholder="Hugging Face")

    btn = gr.Button("Generate brochure PDF", variant="primary")

    md_out = gr.Markdown(label="Brochure (markdown preview)")
    pdf_out = gr.File(label="Download PDF", file_types=[".pdf"])

    btn.click(fn=generate_brochure_pdf, inputs=[url_in, name_in], outputs=[md_out, pdf_out])

# Allow Gradio to serve PDFs saved under PROJECT_ROOT/brochures
brochure_app.launch(allowed_paths=[str(PROJECT_ROOT / "brochures")], share=True)

* Running on local URL:  http://127.0.0.1:7872
* Running on public URL: https://cac71b3a0945914fa1.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
